In [1]:
manufacturer = 'AGILITY'
type_code = '5562'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()
list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url, part_number in zip(list_url, list_part_number):
    work.put_nowait((url, part_number))

def crawler():
    global df
    
    while not work.empty():
        url, part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                if len(html.xpath('span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]/text()')) != 0 and html.xpath('span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]/text()')[0] == 'No Parts Found':
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Url': url}])
                    
                    status = 'ok'
                    
                    break

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                number_index = np.where(np.array(list_part_number) == part_number)[0]
                
                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]
                
                # = = = = = = = = = = = = = = =
                
                index = list(np.intersect1d(number_index, manufacturer_index))
                
                # = = = = = = = = = = = = = = =
                
                if len(index) == 1:
                    index = index[0]
                    
                    # = = = = = = = = = = = = = = =
                    
                    note_1 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                    if len(note_1) == 0:
                        note_1 = ''
                    else:
                        note_1 = note_1[0].strip()
   
                    # = = = = = = = = = = = = = = =
                    
                    note_2 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]/span/span/text()')
                    if len(note_2) == 0:
                        note_2 = ''
                    else:
                        note_2 = [_.strip() for _ in note_2]
                        while '' in note_2:
                            note_2.remove('')
                        note_2 = '\n'.join(note_2)

                    # = = = = = = = = = = = = = = =
                    
                    oe = list_part[index].xpath('./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]')
                    if len(oe) == 0:
                        oe = ''
                    else:
                        oe = oe[0].xpath('./text()')[0].strip().replace(', ', ';')
                        
                    # = = = = = = = = = = = = = = =
                    
                    src = list_part[index].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                    if len(src) == 0:
                        src = ''
                    else:
                        src = 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')
    
                    # = = = = = = = = = = = = = = =
                    
                    part_json = list_part[index].xpath('./descendant::input[contains(@id, "listing_data_essential")]')
                    if len(part_json) == 0:
                        part_key = ''
                    else:
                        part_key = json.loads(part_json[0].xpath('./@value')[0])['partkey'].strip()
                        
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             '序号': '',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Vehcile': '',
                                             'Alternate_OE_Part_Numbers': oe,
                                             'Note_1': note_1,
                                             'Note_2': note_2,
                                             'Pic': '',
                                             'Url': url,
                                             'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + type_code + '&pk=' + part_key,
                                             'Src': src,
                                             'Type_Code': type_code,
                                             'Part_Key': part_key}])
                
                    status = 'ok'
                
                    break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Manufacturer': manufacturer,
                                     'Part_Number': part_number,
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：364

3310003  <->  [ok] - 剩余数量：344 - 当前时间：17:32:51
3310023  <->  [ok] - 剩余数量：343 - 当前时间：17:32:51
3310006  <->  [ok] - 剩余数量：342 - 当前时间：17:32:51
3310015  <->  [ok] - 剩余数量：341 - 当前时间：17:32:52
3310026  <->  [ok] - 剩余数量：340 - 当前时间：17:32:52
3310009  <->  [ok] - 剩余数量：339 - 当前时间：17:32:52
3310013  <->  [ok] - 剩余数量：338 - 当前时间：17:32:52
3310016  <->  [ok] - 剩余数量：337 - 当前时间：17:32:52
3310004  <->  [ok] - 剩余数量：336 - 当前时间：17:32:52
3310010  <->  [ok] - 剩余数量：335 - 当前时间：17:32:52
3310019  <->  [ok] - 剩余数量：334 - 当前时间：17:32:52
3310025  <->  [ok] - 剩余数量：333 - 当前时间：17:32:52
3310008  <->  [ok] - 剩余数量：332 - 当前时间：17:32:52
3310011  <->  [ok] - 剩余数量：331 - 当前时间：17:32:52
3310007  <->  [ok] - 剩余数量：330 - 当前时间：17:32:53
3310029  <->  [ok] - 剩余数量：329 - 当前时间：17:32:53
3310028  <->  [ok] - 剩余数量：328 - 当前时间：17:32:53
3310037  <->  [ok] - 剩余数量：327 - 当前时间：17:32:53
3310035  <->  [ok] - 剩余数量：326 - 当前时间：17:32:54
3310042  <->  [ok] - 剩余数量：325 - 当前时间：17:32:54
3310022  <->  [ok] - 剩余数量：324 - 当前时间：17:32:54
3310049  <->  [ok] - 剩余数量

3310258  <->  [ok] - 剩余数量：165 - 当前时间：17:33:25
3310263  <->  [ok] - 剩余数量：164 - 当前时间：17:33:25
3310250  <->  [ok] - 剩余数量：163 - 当前时间：17:33:25
3310266  <->  [ok] - 剩余数量：162 - 当前时间：17:33:25
3310267  <->  [ok] - 剩余数量：161 - 当前时间：17:33:26
3310268  <->  [ok] - 剩余数量：160 - 当前时间：17:33:26
3310264  <->  [ok] - 剩余数量：159 - 当前时间：17:33:26
3310270  <->  [ok] - 剩余数量：158 - 当前时间：17:33:26
3310272  <->  [ok] - 剩余数量：157 - 当前时间：17:33:27
3310274  <->  [ok] - 剩余数量：156 - 当前时间：17:33:27
3310273  <->  [ok] - 剩余数量：155 - 当前时间：17:33:27
3310246  <->  [ok] - 剩余数量：154 - 当前时间：17:33:28
3310276  <->  [ok] - 剩余数量：153 - 当前时间：17:33:28
3310209  <->  [ok] - 剩余数量：152 - 当前时间：17:33:28
3310278  <->  [ok] - 剩余数量：151 - 当前时间：17:33:28
3310281  <->  [ok] - 剩余数量：150 - 当前时间：17:33:29
3310247  <->  [ok] - 剩余数量：149 - 当前时间：17:33:29
3310202  <->  [ok] - 剩余数量：148 - 当前时间：17:33:29
3310279  <->  [ok] - 剩余数量：147 - 当前时间：17:33:29
3310282  <->  [ok] - 剩余数量：146 - 当前时间：17:33:29
3310041  <->  [ok] - 剩余数量：145 - 当前时间：17:33:29
3310269  <->  [ok] - 剩余数量：144 - 当前

3310729  <->  [ok] - 剩余数量：0 - 当前时间：17:34:09
3310223  <->  [ok] - 剩余数量：0 - 当前时间：17:36:51
3310314  <->  [ok] - 剩余数量：0 - 当前时间：17:37:03

爬虫输出中

爬虫结束
